# Mozzi in Jupyterlab

## Introduction

Currently your Arduino can only beep like a microwave oven. Mozzi brings your Arduino to life by allowing it to produce much more complex and interesting growls, sweeps and chorusing atmospherics. These sounds can be quickly and easily constructed from familiar synthesis units like oscillators, delays, filters and envelopes.

The development of an Arduino Sketch is quite tedious, first you need to wire the processor up to an audio circuit, you need to write your sketch, then you need to upload it to the Microprocessor.

We wanted to simplify this cycle so that you can interactively experiment with your code and get an immediate audible result: We can run the __Arduino Mozzi__ algorythmic music project in Jupyterlab!

## Pulseaudio

The current implementation relys on Pulseaudio. Make sure that you have Pulseaudio installed and that in the __default.pa__ configuration file, the entry __load-module module-native-protocol-tcp__ is uncommented! Finally you need to make sure that the service is started.

For OS/X you install it with ```brew install pulseaudio```. The default.pa file can be found in ```/usr/local/opt/pulseaudio/etc/pulse``` and the service is started with ```brew services restart pulseaudio```

## Initialization

In order to initizlize the environment, we need to define the include paths, library paths and load the dynamic libraries first:

In [49]:
#pragma cling add_include_path("/opt/mozzi/include/arduino")
#pragma cling add_include_path("/opt/mozzi/include/mozzi")
#pragma cling add_library_path("/opt/mozzi/lib")
#pragma cling load("libarduino_emulator")
#pragma cling load("libmozzi")


Mozzi also requires the difnition of the Arduino Version, so we provide 200:

In [58]:
#define ARDUINO 200

Now we are ready to include the Arduino.h and the Mozzi.h header files. This makes the functionality visible:

In [59]:
#include "Arduino.h"
#include "Mozzi.h"


We can use regular Arduino commands now:

In [60]:
Serial.println("hallo...");

hallo...


but we are more interrested in Mozzi functionality: We can check some defined values:

In [61]:
CONTROL_RATE

64

In [62]:
AUDIO_RATE

32768

## Defining a Sketch

We use the output of a sin function as demo, how to set up a sketch. We have the __full Mozzi functionality__ available, but since we have an interactive environment, the overall control flow is a little bit different.

The API functions are made available with the Mozzi object:

In [51]:
Mozzi.controlRate()

64

In [52]:
Mozzi.channels()

1

In [53]:
Mozzi.audioRate()

32768

In [63]:
Mozzi.mozziMicros()

0

As a side note, I am using the __new object API__, the old API with public global methods is available as well, but it only supports the basic functionality and not the new 'interactive' functionality:

In [67]:
mozziMicros()

0

We can define new objects: Here we set up a new Oscil. 

In [37]:
#include <Oscil.h> // oscillator template
#include <tables/sin2048_int8.h> // sine table for oscillator// use: Oscil <table_size, update_rate> oscilName (wavetable), look in .h file of table #included above

// use: Oscil <table_size, update_rate> oscilName (wavetable), look in .h file of table #included above
Oscil <SIN2048_NUM_CELLS, AUDIO_RATE> aSin(SIN2048_DATA);


and we need to define an updateControl and an updateAudio method. Unlike in standard Mozzi you are free to use any name you want. You could even use static methods as part of a class or lambda expressions. 

Here we just stick to the Mozzi Arduiono standard:

In [38]:
void updateControl(){
  // put changing controls in here
}


In [39]:
int16_t updateAudio(){
  return MonoOutput::from8Bit(aSin.next()); // return an int signal centred around 0
}


Unlike in Arduino we could just execute the setup logic outside of a method, but to be closer to what we are used to do, we create a __setup__ method. 
We __need to provide the updateAudio and updateControl in the Mozzi.start__ method. We could also provide the optional audio update rate as 3d parameter, if we are not happy with the default value.

In [68]:
void setup(){
    Mozzi.start(updateAudio, updateControl);
    aSin.setFreq(440); // set the frequency
}


Finally, we execute our setup() method ourselfs - we do not need to implment a loop() method! 

In [70]:
setup();

and to hear some sound we use the __play__ method: it expects the number of seconds to play the generated tone (as optional argument): 

In [27]:
Mozzi.play(10)

true